In [24]:
import os
import re
import math
import pandas as pd
import numpy as np

In [25]:
def ecef2enu(x, y, z, lat, long, h, xr, yr, zr) :
    e = 0.0810820288
    pi = math.radians(lat)       #degree 단위를 radian로 변경
    lamb = math.radians(long)
        
    k = 1-(e**2)*(math.sin(pi)**2)
    pt = 6378000/(math.sqrt(k))  #지구 적도 반지름 6,378km, 극 반지름 6,357km
    pm = pt*(1-e**2)
    
    RMatrix = np.matrix([[-math.sin(lamb), math.cos(lamb), 0],
                         [-math.sin(pi)*math.cos(lamb), -math.sin(pi)*math.sin(lamb), math.cos(pi)], 
                         [math.cos(pi)*math.cos(lamb), math.cos(pi)*math.sin(lamb), math.sin(pi)]])
    
    Matrix = np.array([[x-xr], [y-yr], [z-zr]])
    enu = np.matmul(RMatrix, Matrix)
    return enu.flatten()

In [26]:
df = pd.read_csv("C:\\Users\\juyeo\\Desktop\\GNSS_DATA\\_ECEF\\HOMI_.csv")
df.head()

,x-ecef(m),y-ecef(m),z-ecef(m)
0,-3287391.334,3978484.594,3735219.075
1,-3287391.396,3978484.645,3735219.128
2,-3287391.388,3978484.621,3735219.134
3,-3287391.387,3978484.619,3735219.100
4,-3287391.383,3978484.622,3735219.122


In [27]:
df2 = df.astype(float)
true_ecef = [-3287391.376, 3978484.605, 3735219.106]
true_llh = [36.07804508, 129.56666192, 42.323]
X = df['x-ecef(m)']
Y = df['y-ecef(m)']
Z = df['z-ecef(m)']
print(true_llh)
err_enu = np.empty((0,3), float)
df.info()

[36.07804508, 129.56666192, 42.323]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2824 entries, 0 to 2823
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   x-ecef(m)  2824 non-null   float64
 1   y-ecef(m)  2824 non-null   float64
 2   z-ecef(m)  2824 non-null   float64
dtypes: float64(3)
memory usage: 66.3 KB


In [28]:
for i in range(2824) :
    err = ecef2enu(X[i], Y[i], Z[i], true_llh[0], true_llh[1], true_llh[2], true_ecef[0], true_ecef[1], true_ecef[2])
    err_enu = np.append(err_enu, np.array(err), axis=0)
df['err_e'] = err_enu[:, 0]
df['err_n'] = err_enu[:, 1]
df['err_u'] = err_enu[:, 2]

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2824 entries, 0 to 2823
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   x-ecef(m)  2824 non-null   float64
 1   y-ecef(m)  2824 non-null   float64
 2   z-ecef(m)  2824 non-null   float64
 3   err_e      2824 non-null   float64
 4   err_n      2824 non-null   float64
 5   err_u      2824 non-null   float64
dtypes: float64(6)
memory usage: 132.5 KB


In [30]:
rms_e = np.sqrt(np.mean(df['err_e']**2))
rms_n = np.sqrt(np.mean(df['err_n']**2))
rms_u = np.sqrt(np.mean(df['err_u']**2))
print(rms_e)
print(rms_n)
print(rms_u)

0.02971354677941083
0.021288878632798793
0.05679182056201781


In [31]:
df.to_csv("C:\\Users\\juyeo\\Desktop\\GNSS_DATA\\_ECEF\\YODK_ecef_2017_rms.csv")